In [1]:
# import utility functions
import utils
from utils.flow_models import *
from utils.target import *
from utils.training import *

# import pytorch
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.distributions.transforms as transform
# set number of threads
torch.set_num_threads(8)

# numerical libs
import scipy
import scipy.io

# import plotting
import matplotlib.pyplot as plt
%matplotlib inline

# import I/O
import os
import sys

# set seed
seed = 2
np.random.seed(seed)
torch.manual_seed(0)

## TF + ResNet

In [ ]:
# load tensorizing flow dataset
tf_dataset = utils.datasets.TensorizingFlowDataset("pathsampling_fullrank_small.mat", gaussian_data=True)
# initialize NF model
tpt_tf = NormalizingFlow(dim=tf_dataset.dim, blocks=RESNET_PATH, \
                            flow_length=3)

# begin training
gl1d_anti_tf_report = train(tf_dataset, tpt_tf, tpt_logpdf, 
          num_epochs=200,
          batch_size=2**9,
          verbose=True,
          lr=1e-5, 
          use_scheduler=True,
          grad_clip=1e+4, 
          schedule_rate=0.999)

### Full Rank Statistics

In [ ]:
# load data
fullrank_data = scipy.io.loadmat("./data/pathsampling_fullrank_small.mat")

In [ ]:
log_prior = torch.tensor(fullrank_data['prior_logpdf'], requires_grad=True)
data = torch.tensor(fullrank_data['X'].T, requires_grad=True)

In [ ]:
kl_loss = torch.mean((log_prior + path_action(data).reshape(-1, 1))).item()
kl_loss

In [ ]:
# mean path 
mean_path = data.mean(0).detach().numpy()
std_path = data.std(0).detach().numpy()

In [ ]:
# save data
scipy.io.savemat("./utils/full_rank_stats9.mat", {'tpt': {'mean': mean_path, 'std': std_path, \
                                                      'loss': kl_loss}})

In [ ]:
scipy.io.loadmat("./utils/full_rank_stats9.mat")['tpt']['loss'][0][0][0]

### Other Testing